In [3]:
# If you want the figures to appear in the notebook, 
# and you want to interact with them, use
# %matplotlib notebook

# If you want the figures to appear in the notebook, 
# and you don't want to interact with them, use
# %matplotlib inline

# If you want the figures to appear in separate windows, use
# %matplotlib qt5

# tempo switch from one to another, you have to select Kernel->Restart

%matplotlib inline

from modsim import *
import math

In [10]:
sat_5 = System(v0=0, vExhaust=2550, mVehicle=239725, mDot=1169.36)

In [33]:
def calcVfinal(mFuel):
    
    V = sat_5.v0
    vE = sat_5.vExhaust
    mV = sat_5.mVehicle
    dm = sat_5.mDot
    m0 = mFuel
    
    # Time to use up all the fuel
    t = mFuel/dm
    V_time =  V + vE * math.log((mV + mFuel)/(mV)) - 9.8 * t
    
    
    while ((mFuel - dm) > 0):
        # LOG THIS INTO A TIMESERIES
        V = V + vE * math.log((mV + mFuel)/(mV + mFuel - dm)) - 9.8
        mFuel -= dm
      
    print(V_time)
    print(V)
    
    # return V
    
    
    

In [34]:
calcVfinal(2527188)

-14942.1973306
-14942.6500883


In [32]:
def simulate_saturn_v():
    
    # Stage 1
    m0 = 2900000
    mf = 731000
    mFuel = m0-mf
    vE = 263*9.8
    dm = 12890
    
    t = mFuel/dm
    dV1 =  vE * math.log(m0/mf) - 9.8 * t
    
    # Stage 2
    m0 = 600000
    mf = 156000
    mFuel = m0-mf
    vE = 421*9.8
    dm = mFuel/475
    
    t = mFuel/dm
    dV2 =  vE * math.log(m0/mf) - 9.8 * t
    
    # Stage 3
    m0 = 120800
    mf = 10000
    mFuel = m0-mf
    vE = 421*9.8
    dm = mFuel/475
    
    t = mFuel/dm
    dV3 =  vE * math.log(m0/mf) - 9.8 * t
    
    print(dV1)
    print(dV2)
    print(dV3)
    print(dV1+dV2+dV3)
    
simulate_saturn_v()

1902.7468864975515
902.7564567806376
5624.641910044014
8430.145253322204


In [30]:
def simulate_apollo_cmd_module():
    
    m0 = 14690
    mf = 11900   # Actual empty value is 11900, but I estimate ~1/2 of that fuel was actually burned for this maneuver
    vE = 319*9.8
    
    dV =  vE * math.log(m0/mf)
    print(dV)
    
simulate_apollo_cmd_module()

658.4670982715267


In [ ]:
rocket = (system)

def calcVfinal(mFuel):
    
    V = 0
    vE = 1200*9.8
    mV = sat_5.mVehicle
    dm = sat_5.mDot
    m0 = mFuel
    
    # Time to use up all the fuel
    t = mFuel/dm
    V_time =  V + vE * math.log((mV + mFuel)/(mV)) - 9.8 * t
    
    
    while ((mFuel - dm) > 0):
        #print(vE * math.log((mV + mFuel)/(mV + mFuel - dm)))
        V = V + vE * math.log((mV + mFuel)/(mV + mFuel - dm)) - 9.8
        mFuel -= dm
      
    print(V_time)
    print(V)
    
    # return V